In [1]:
import numpy as np
import os
import torch
from torch import nn, optim
import importlib

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

Using device cuda:0
GeForce GTX 1660 SUPER
P106-100


In [2]:
from torch.utils.data import Dataset, DataLoader
from data.MinicityDataset import TrafficDataset

dbpath = '/home/rong/disk/database/minicity.db'
bs = 12
train_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = True),
                         batch_size = bs, shuffle=True, drop_last = True)
test_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = False),
                          batch_size = bs, shuffle=True, drop_last = True)

# batch = next(iter(train_loader))
# batch['data']

In [3]:
from models.minicity import convVAE

X_dim = 4
traj_size = 25 * 4
z_dim = 50
cnn_out_size = 300
cond_out_size = 300

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)

checkpoint = torch.load('checkpoints/minicity_attention_7.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
print(model)

convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap_pool2): AdaptiveAvg

In [4]:
from utils.MinicityOmpl import getSampleData
from utils.Minicity import plotData, plotOrientSpeed, plotAlpha
# importlib.reload(MinicityOmpl)

test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
# viz_idx = 8753 #弯道
# viz_idx = 143300 #弯道
# viz_idx = 5107 #直行
# viz_idx = 2984 #转盘
# viz_idx = 101538 #转盘
# viz_idx = 153309 #阻塞
# viz_idx = 109779 #变道

print(viz_idx)
startgoal, egoid, occ, y_viz, time_stamp, alpha = getSampleData(model, test_data, viz_idx, device)

%matplotlib
plotData(occ, startgoal, y_viz)
plotOrientSpeed(startgoal, y_viz)
# plotAlpha(alpha)

103952
torch.Size([25, 4])
torch.Size([25, 4])
time_step:  1478.0
start [ 0.      0.     -1.4444  8.7   ]
goal [ 10.18   -13.99    -0.6669   5.3   ]
Using matplotlib backend: Qt5Agg
[  0.       0.      -1.4444   8.7     10.18   -13.99    -0.6669   5.3   ]
133.93333435058594 146.6333325703939


In [5]:
plotAlpha(alpha)

# commonroad collision

In [6]:
import os
from commonroad.common.file_reader import CommonRoadFileReader

file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/minicity.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()
from utils.MinicityOmpl import show_scenario
show_scenario(scenario, 0)

In [7]:
from commonroad.scenario.scenario import Scenario
from commonroad_cc.collision_detection.pycrcc_collision_dispatch import create_collision_checker, create_collision_object
import pycrcc
from utils.MinicityOmpl import create_roadcc
carcc = create_collision_checker(scenario)
roadcc = create_roadcc(scenario)

## 碰撞测试

In [8]:
import numpy as np
from commonroad.scenario.trajectory import State, Trajectory
from commonroad.prediction.prediction import TrajectoryPrediction
from commonroad.geometry.shape import Rectangle
from commonroad.scenario.obstacle import StaticObstacle
from commonroad_cc.collision_detection.pycrcc_collision_dispatch import create_collision_object

# # create a trajectory for the ego vehicle starting at time step 0
time_start = 0
position = np.array([[153, 147]])
orientation = [0]
state_list = list()
for k in range(0, len(position)):
    state_list.append(State(**{'position': position[k], 'orientation':orientation[k]}))
trajectory = Trajectory(time_start, state_list)
# create the shape of the ego vehicle
shape = Rectangle(length=3.9, width=1.9)
# # create a TrajectoryPrediction object consisting of the trajectory and the shape of the ego vehicle
traj_pred = TrajectoryPrediction(trajectory=trajectory, shape=shape)
# create a collision object using the trajectory prediction of the ego vehicle
co = create_collision_object(traj_pred)

# test the trajectory of the ego vehicle for collisions
print('Collision between the trajectory of the ego vehicle and objects in the environment: ', carcc.collide(co))
print('Collision between the trajectory of the ego vehicle and road: ', roadcc.collide(co))
print('Vallid', roadcc.collide(co) and not carcc.collide(co))

Collision between the trajectory of the ego vehicle and objects in the environment:  False
Collision between the trajectory of the ego vehicle and road:  True
Vallid True


# ompl

In [9]:
from ompl import base as ob
from ompl import control as oc
from ompl import app as oa

class ValidityChecker(ob.StateValidityChecker):
    def __init__(self,  si, ccc, ccr, time_stamp):
        super(ValidityChecker, self).__init__(si)
        self.ccc = ccc
        self.ccr = ccr
        self.time_stamp = int(time_stamp)
        self.count = 0
        self.collision_count = 0
        self.states_ok = []
        self.states_bad = []
        # Returns whether the given state's position overlaps the
        # circular obstacle
    def isValid(self, state):
        time_start = self.time_stamp
        position = np.array([[state[0].getX(), state[0].getY()]])
        orientation = [state[0].getYaw()]
        state_list = list()
        for k in range(0, len(position)):
            state_list.append(State(**{'position': position[k], 'orientation':orientation[k]}))
        trajectory = Trajectory(time_start, state_list)
#         shape = Rectangle(length=3.9, width=1.9)
        shape = Rectangle(length=1, width=0.5)
        traj_pred = TrajectoryPrediction(trajectory=trajectory, shape=shape)
        co = create_collision_object(traj_pred)

        result = self.ccr.collide(co) and not self.ccc.collide(co)
        self.count += 1
#         print("count", self.count)
        if not result:
            self.collision_count += 1
#             print("collision", self.collision_count)
#             self.states_bad.append(sample)
#         else:
#             self.states_ok.append(sample)
        return result
    
    def clearcount(self):
        self.count = 0
        self.collision_count = 0

car = oa.DynamicCarPlanning()
mychecker = ValidityChecker(car.getSpaceInformation(), carcc, roadcc, 0)
print(mychecker)

In [10]:
from ompl import util as ou
import math
from utils.MinicityOmpl import warp2pi


class MyStateSampler(ob.StateSampler):
    def __init__(self, space, cvae_samples):
        super(MyStateSampler, self).__init__(space)
        self.space = space
        print(self.space)
        self.cvae_samples = cvae_samples
        self.cvae_num = cvae_samples.shape[0]
        self.i = 0
        self.rng_ = ou.RNG()
        print("my state sampler")
        
        boundse2 = self.space.getSubspace(0).getBounds()
        bound2 = self.space.getSubspace(1).getBounds()
        self.xlow = boundse2.low[0] + 140
        self.xhigh = boundse2.high[0] - 140
        self.ylow = boundse2.low[1] + 140
        self.yhigh = boundse2.high[1] - 140
        
        self.vlow = bound2.low[0]
        self.vhigh = bound2.high[0]
        self.vanglelow = bound2.low[1]
        self.vanglehigh = bound2.high[1]
        
#     def __call__(self, i): 
#         return self
        
    def sampleUniform(self, state):
        chance = self.rng_.uniformReal(0, 1)
#         print("my sample uniform")
        
        if chance < 0.8:
            x = self.cvae_samples[self.i, 0]
            y = self.cvae_samples[self.i, 1]
            yaw = self.cvae_samples[self.i, 2]
            yaw = warp2pi(yaw)
            v = self.cvae_samples[self.i, 3]
            self. i += 1
            if self.i == self.cvae_num:
                self.i = 0
            print("here ", self.i)
        else:
            x = self.rng_.uniformReal(self.xlow, self.xhigh)
            y = self.rng_.uniformReal(self.ylow, self.yhigh)
            yaw = self.rng_.uniformReal(-math.pi, math.pi)
            v = self.rng_.uniformReal(self.vlow, self.vhigh)
            
        vangle = self.rng_.uniformReal(self.vanglelow, self.vanglehigh)
        state[0].setX(float(x))
        state[0].setY(float(y))
        state[0].setYaw(float(yaw))
        state[1][0] = float(v)
        state[1][1] = float(vangle)
        
        return True
    
    def sampleUniformNear(self, state, nearstate, dis):
        print("sample uniform near")
        self.sampleUniform(state)
        
    def sampleGaussian(self, state, nearstate, dis):
        print("sample guassian")
        self.sampleUniform(state)
        
# return an instance of my sampler
def allocMyValidStateSampler(si):
    return MyValidStateSampler(si, y_viz)

def allocMyStateSampler(space):
    return MyStateSampler(space, y_viz)

In [11]:
import sys
from os.path import abspath, dirname, join

from ompl import base as ob
from ompl import control as oc
from ompl import app as oa

from commonroad_cc.visualization.draw_dispatch import draw_object
import matplotlib.pyplot as plt

# from utils.MinicityOmpl import checkpath

def plt_ompl_result(path, time_step):
    if path:
        fig1 = plt.figure(figsize=(10, 6), dpi=80)
        ax1 = fig1.add_subplot(111, aspect='equal')   
        draw_params = {}
        draw_params['time_begin'] = time_step
        draw_object(scenario, draw_params=draw_params)

        path.interpolate(); # uncomment if you want to plot the path

        states = path.getStates()
        for state in states:
            state = state[0]
            plt.scatter(state.getX(), state.getY(), color="green", s=250, edgecolors='black')  # path
        #     for state in collisionchecker.states_ok:
        #         plt.scatter(state[0], state[1], color="green", s=100, edgecolors='green')  # free sample
        #     for state in collisionchecker.states_bad:
        #         plt.scatter(state[0], state[1], color="red", s=100, edgecolors='red')  # collision sample
        # plt.scatter(start().getX(), start().getY(), color="blue", s=250, edgecolors='black')  # init
        # plt.scatter(goal().getX(), goal().getY(), color="red", s=250, edgecolors='black')  # goal

        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()
    else:
        print("no path to plot")

def plttree(planner, startgoal):
    fig1 = plt.figure(figsize=(10, 6), dpi=80)
    ax1 = fig1.add_subplot(111, aspect='equal')
    
    plannerdata = ob.PlannerData(planner.getSpaceInformation())
    planner.getPlannerData(plannerdata)

    print("num edges: ", plannerdata.numEdges())
    print("num vertices: ", plannerdata.numVertices())

    plt.scatter(startgoal[0], startgoal[1], color="blue", s=250, edgecolors='black')  # init
    plt.scatter(startgoal[4], startgoal[5], color="red", s=250, edgecolors='black')  # goal

    num_ver = plannerdata.numVertices()
    for i in range(0, num_ver):
        for j in range(i, num_ver):
            if plannerdata.edgeExists(i, j):
                plt.plot([plannerdata.getVertex(i).getState()[0].getX(), plannerdata.getVertex(j).getState()[0].getX()], [plannerdata.getVertex(i).getState()[0].getY(), plannerdata.getVertex(j).getState()[0].getY()], color='gray')
                
        plt.scatter( plannerdata.getVertex(i).getState()[0].getX(), plannerdata.getVertex(i).getState()[0].getY(), color="green", s=50, edgecolors='black')  # vertice

    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [12]:
import math

def dynamicCarMinicity(setup, basepose, startgoal, sampleallocator):
    print("\n\n***** Planning for a %s *****\n" % setup.getName())
    # plan for dynamic car in SE(2)
    stateSpace = setup.getStateSpace()
    sizebound = 200
    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0, basepose[0] - sizebound)
    bounds.setHigh(0, basepose[0] + sizebound)
    bounds.setLow(1, basepose[1]- sizebound)
    bounds.setHigh(1, basepose[1] + sizebound)
    stateSpace.getSubspace(0).setBounds(bounds)
    bounds.setLow(0, -9)
    bounds.setHigh(0, 25)
#     bounds.setLow(1, -0.52)
#     bounds.setHigh(1, 0.52)
    bounds.setLow(1, -2)
    bounds.setHigh(1, 2)
    stateSpace.getSubspace(1).setBounds(bounds)
    print(setup.getStateSpace().getSubspace(0).getBounds().high[0])
    bounds.setLow(0, -.5)
    bounds.setHigh(0, 2.5)
    bounds.setLow(1, -math.pi * 40/180)
    bounds.setHigh(1, math.pi * 40/180)
    controlSpace = setup.getControlSpace()
    controlSpace.setBounds(bounds)
    print(setup.getControlSpace().getBounds().high[0])

    # define start state
    start = ob.State(stateSpace)
    start[0] = float(startgoal[0])
    start[1] = float(startgoal[1])
    start[2] = float(startgoal[2])
    start[3] = float(startgoal[3])
    start[4] = 0

    print(start)
    # define goal state
    goal = ob.State(stateSpace)
    goal[0] = float(startgoal[4])
    goal[1] = float(startgoal[5])
    goal[2] = float(startgoal[6])
    goal[3] = float(startgoal[7])
    goal[4] = 0
    print(goal)

    # set the start & goal states
    setup.setStartAndGoalStates(start, goal, .5)
    setup.getSpaceInformation().setStateValidityChecker(mychecker)
    setup.getSpaceInformation().getStateSpace().setStateSamplerAllocator(ob.StateSamplerAllocator(sampleallocator))
#     print(setup.getSpaceInformation().getStateValidityChecker())
    planner = oc.RRT(setup.getSpaceInformation())

    
    setup.setPlanner(planner)
#     print(planner.getSpaceInformation().getStateValidityChecker())
    planner.setProblemDefinition(setup.getProblemDefinition())
#     setup.getPlanner().setup()
    # try to solve the problem
#     planner.setNumSamples(30)
    plan_res =  planner.solve(0.2)
    path = planner.getProblemDefinition().getSolutionPath()
    if plan_res:
#         path.interpolate(); # uncomment if you want to plot the path
        print(path.printAsMatrix())
        if plan_res.asString() != 'Exact solution':
            print("Solution is approximate. Distance to actual goal is %g" %
                  planner.getProblemDefinition().getSolutionDifference())
    else:
        print("plan fail")
    return path, planner

In [13]:
from utils.MinicityOmpl import getproblem

test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
# viz_idx = 8753 #弯道
# viz_idx = 143300 #弯道
# viz_idx = 5107 #直行
# viz_idx = 2984 #转盘
# viz_idx = 101538 #转盘
# viz_idx = 153309 #阻塞
# viz_idx = 109779 #变道

print(viz_idx)
startgoal, egoid, occ, y_viz, time_stamp, alpha = getSampleData(model, test_data, viz_idx, device)
%matplotlib
plotData(occ, startgoal, y_viz)

ego = scenario.obstacle_by_id(egoid)
basepose, startgoal = getproblem(scenario, egoid, time_stamp, startgoal)

135372
torch.Size([25, 4])
torch.Size([25, 4])
time_step:  376.0
start [ 0.    0.   -0.    5.25]
goal [ 8.439999  0.       -0.        5.23    ]
Using matplotlib backend: Qt5Agg
[ 0.        0.       -0.        5.25      8.439999  0.       -0.
  5.23    ]
128.13332875569662 100.0


In [14]:
car = oa.DynamicCarPlanning()
car.getSpaceInformation().setPropagationStepSize(0.01)
car.getSpaceInformation().setStateValidityCheckingResolution(0.001)
mychecker = ValidityChecker(car.getSpaceInformation(), carcc, roadcc, 0)
car.getSpaceInformation().setStateValidityChecker(mychecker)
# scenario.remove_obstacle(ego)
path, planner = dynamicCarMinicity(car, basepose, startgoal, allocMyStateSampler)
# scenario.add_objects(ego)
print("Count", mychecker.count)
print("Collision", mychecker.collision_count)
plt_ompl_result(path, int(time_stamp))
plttree(planner, startgoal)
print("Cost", path.length())
print("Distance", planner.getProblemDefinition().getSolutionDifference())



***** Planning for a Dynamic car *****

235.86
2.5
Compound state [
Compound state [
RealVectorState [35.86 178.4]
SO2State [-0]
]
RealVectorState [5.25 0]
]

Compound state [
Compound state [
RealVectorState [44.3 178.4]
SO2State [-0]
]
RealVectorState [5.23 0]
]

my state sampler
here  1
here  2
here  3
here  4
here  5
here  6
here  7
here  8
here  9
here  10
here  11
here  12
here  13
here  14
here  15
here  16
here  17
here  18
here  19
here  20
here  21
here  22
here  23
here  24
here  25
here  26
here  27
here  28
here  29
here  30
here  31
here  32
here  33
here  34
here  35
here  36
here  37
here  38
here  39
here  40
here  41
here  42
here  43
here  44
here  45
here  46
here  47
here  48
here  49
here  50
here  51
here  52
here  53
here  54
here  55
here  56
here  57
here  58
here  59
here  60
here  61
here  62
here  63
here  64
here  65
here  66
here  67
here  68
here  69
here  70
here  71
here  72
here  73
here  74
here  75
here  76
here  77
here  78
here  79
here  80
here

In [29]:
def getDataNosample(model, test_data, viz_idx, device):
    batch = test_data[viz_idx]
    startgoal = torch.from_numpy(batch["start_goal"]).to(device)
    egoid = batch["egoid"]
    time_stamp = batch["timeprob"]
    print("time_step: ", time_stamp)

    startgoal=startgoal.cpu().detach().numpy() * 50
    print("start", startgoal[0:4])
    print("goal", startgoal[4:8])

    return startgoal, egoid, time_stamp

from statistics import mean 

def rollout(test_data, viz_idx, mychecker, allocstatesampler):
    if type(allocstatesampler) == allocMyStateSampler:
        startgoal, egoid, occ, y_viz, time_stamp, alpha = getSampleData(model, test_data, viz_idx, device)
    else:
        startgoal, egoid,  time_stamp= getDataNosample(model, test_data, viz_idx, device)
        
#     %matplotlib
    ego = scenario.obstacle_by_id(egoid)
    basepose, startgoal = getproblem(scenario, egoid, time_stamp, startgoal)
    print("startgoal", startgoal)
    print("basepose", basepose)
    car = oa.DynamicCarPlanning()
    car.getSpaceInformation().setPropagationStepSize(0.01)
    car.getSpaceInformation().setStateValidityCheckingResolution(0.001)
    car.getSpaceInformation().setStateValidityChecker(mychecker)
#     scenario.remove_obstacle(ego)
    costs = []
    distances = []
    collisions = []
    counts = []
    for i in range(0, 10):
        mychecker.clearcount()
        path, planner = dynamicCarMinicity(car, basepose, startgoal, allocstatesampler)
        distance = planner.getProblemDefinition().getSolutionDifference()
        count = mychecker.count
        collision = mychecker.collision_count    
        if path and distance < 5:
            length = path.length()
        else:
            length = 10
        costs.append(length)
        distances.append(distance)
        counts.append(count)
        collisions.append(collision)
        
#     scenario.add_objects(ego)
    print("Count", mean(counts))
    print("Collision", mean(collisions))
    plt_ompl_result(path, int(time_stamp))
    plttree(planner, startgoal)
    print("Distance", mean(distances))

    return mean(costs), mean(collisions), mean(distances)

test_data = test_loader.dataset
costs_att = []
collisions_att = []
distance2goals_att = []
costs_rand = []
collisions_rand = []
distance2goals_rand = []
import random

for i in range(0, 1):
    viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#     viz_idx = 141858
#     viz_idx = random.choice(viz_my)
#     viz_idx = viz_my[1]
    print(viz_idx)
    cost_my, collision_my, distance2goal_my = rollout(test_data, viz_idx, mychecker, allocMyStateSampler)
    costs_att.append(cost_my)
    collisions_att.append(collision_my)
    distance2goals_att.append(distance2goal_my)
    cost, collision, distance2goal = rollout(test_data, viz_idx, mychecker, allocRandomSampler)
    costs_rand.append(cost)
    collisions_rand.append(collision)
    distance2goals_rand.append(distance2goal)
    if cost_my + distance2goal_my * 0.1 < cost + distance2goal * 0.1 and collision_my < collision :
        viz_my.append(viz_idx)

157220
time_step:  2226.0
start [ 0.      0.      1.2849 13.45  ]
goal [-15.460007   30.630001   -3.6737003  15.700001 ]
startgoal [218.55      25.39       1.2849    13.45     203.09      56.02
   2.609485  15.700001]
basepose [218.55  25.39]


***** Planning for a Dynamic car *****

418.55
2.5
Compound state [
Compound state [
RealVectorState [218.55 25.39]
SO2State [1.2849]
]
RealVectorState [13.45 0]
]

Compound state [
Compound state [
RealVectorState [203.09 56.02]
SO2State [2.60948]
]
RealVectorState [15.7 0]
]

my state sampler
here  1
here  2
here  3
here  4
here  5
here  6
here  7
here  8
here  9
here  10
here  11
here  12
here  13
here  14
here  15
here  16
here  17
here  18
here  19
here  20
here  21
here  22
here  23
here  24
here  25
here  26
here  27
here  28
here  29
here  30
here  31
here  32
here  33
here  34
here  35
here  36
here  37
here  38
here  39
here  40
here  41
here  42
here  43
here  44
here  45
here  46
here  47
here  48
here  49
here  50
here  51
here  52


In [19]:
len(viz_my)

1

In [17]:
viz_my = []

In [ ]:
viz_my

In [30]:
from statistics import mean 
print("length", len(costs_att))
# print(costs_att)
print("average cost", mean(costs_att))
# print(distance2goals_att)
print("average distance2gaol", mean(distance2goals_att))
# print(collisions_att)
print("average collisions", mean(collisions_att))
# for i in len()

length 1
average cost 10
average distance2gaol 20.443017790276173
average collisions 11.4


In [31]:
'''''''# print(costs_rand)'''''''
print("average cost", mean(costs_rand))
# print(distance2goals_rand)
print("average distance2gaol", mean(distance2goals_rand))
# print(collisions_rand)
print("average collisions",mean(collisions_rand))


average cost 10
average distance2gaol 14.76027017881045
average collisions 87


In [16]:
from ompl import util as ou
import math
from utils.MinicityOmpl import warp2pi


class RandomSampler(ob.StateSampler):
    def __init__(self, space, cvae_samples):
        super(RandomSampler, self).__init__(space)
        self.space = space
        print(self.space)
        self.cvae_samples = cvae_samples
        self.cvae_num = cvae_samples.shape[0]
        self.i = 0
        self.rng_ = ou.RNG()
        print("my state sampler")
        
        boundse2 = self.space.getSubspace(0).getBounds()
        bound2 = self.space.getSubspace(1).getBounds()
        self.xlow = boundse2.low[0]
        self.xhigh = boundse2.high[0]
        self.ylow = boundse2.low[1]
        self.yhigh = boundse2.high[1]
        
        self.vlow = bound2.low[0]
        self.vhigh = bound2.high[0]
        self.vanglelow = bound2.low[1]
        self.vanglehigh = bound2.high[1]
        
#     def __call__(self, i): 
#         return self
        
    def sampleUniform(self, state):
        chance = self.rng_.uniformReal(0, 1)
#         print("my sample uniform")
        
        if chance < 0:
            x = self.cvae_samples[self.i, 0]
            y = self.cvae_samples[self.i, 1]
            yaw = self.cvae_samples[self.i, 2]
            yaw = warp2pi(yaw)
            v = self.cvae_samples[self.i, 3]
            self. i += 1
            if self.i == self.cvae_num:
                self.i = 0
            print("here ", self.i)
        else:
            x = self.rng_.uniformReal(self.xlow, self.xhigh)
            y = self.rng_.uniformReal(self.ylow, self.yhigh)
            yaw = self.rng_.uniformReal(-math.pi, math.pi)
            v = self.rng_.uniformReal(self.vlow, self.vhigh)
            
        vangle = self.rng_.uniformReal(self.vanglelow, self.vanglehigh)
        state[0].setX(float(x))
        state[0].setY(float(y))
        state[0].setYaw(float(yaw))
        state[1][0] = float(v)
        state[1][1] = float(vangle)
        
        return True
    
    def sampleUniformNear(self, state, nearstate, dis):
        print("sample uniform near")
        self.sampleUniform(state)
        
    def sampleGaussian(self, state, nearstate, dis):
        print("sample guassian")
        self.sampleUniform(state)
        
# return an instance of my sampler
def allocRandomStateSampler(si):
    return MyValidStateSampler(si, y_viz)

def allocRandomSampler(space):
    return RandomSampler(space, y_viz)

In [ ]:
car = oa.DynamicCarPlanning()
car.getSpaceInformation().setPropagationStepSize(0.1)
car.getSpaceInformation().setStateValidityCheckingResolution(0.001)
mychecker = ValidityChecker(car.getSpaceInformation(), carcc, roadcc, 0)
car.getSpaceInformation().setStateValidityChecker(mychecker)
# scenario.remove_obstacle(ego)
path, planner = dynamicCarMinicity(car, basepose, startgoal, allocRandomSampler)
# scenario.add_objects(ego)
print("Count", mychecker.count)
print("Collision", mychecker.collision_count)
plt_ompl_result(path, int(time_stamp))
plttree(planner, startgoal)
print("Cost", path.length())